# 组态相互作用

In [2]:
from pyscf import gto, scf, fci, ao2mo
import numpy as np
from functools import reduce

mol = gto.Mole()
mol.atom = [("H",(0,0,0)),("H",(0,0,1))]
mol.basis = "sto3g"
mol.build()

mf = scf.RHF(mol)
mf.run()

converged SCF energy = -1.06610864931794


In [30]:
norb = mol.nao_nr()
nelec = mol.nelectron
ecore = mol.energy_nuc()
mo_coeff = mf.mo_coeff
h1e = reduce(np.dot, (mo_coeff.T, mf.get_hcore(), mo_coeff))
# eri = ao2mo.full(mf.mol, mo_coeff)
eri = ao2mo.kernel(mf.mol, mo_coeff,aosym=1)

In [31]:
myci = fci.direct_spin1.FCI(mf)
e, v = myci.kernel(h1e,eri,norb,nelec,ecore = ecore)
e

-1.1011503302326187

In [33]:
eri.shape

(4, 4)

In [24]:
from pyscf.fci import fci_slow
fci_slow.kernel(h1e,eri,norb,nelec,ecore = ecore)

-1.101150330232619

In [58]:
import numpy as np
from pyscf import gto, scf, ao2mo
import itertools
import bisect

# 1. 定义分子并进行HF计算
mol = gto.M(atom='H 0 0 0; H 0 0 1.0', basis='sto-3g')
mf = scf.RHF(mol).run()

# 2. 获取分子轨道积分
h1e = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)  # 单电子积分
nmo = h1e.shape[0]
# eri = ao2mo.kernel(mol, mf.mo_coeff)  # 双电子积分 (物理 notation)
# eri = eri.reshape(nmo, nmo, nmo, nmo)
eri = ao2mo.kernel(mf.mol, mo_coeff,aosym=1)
eri = eri.reshape(nmo, nmo, nmo, nmo)
# 3. 扩展到自旋轨道
nspin = nmo * 2  # 自旋轨道数目

# 单电子积分扩展到自旋轨道
h1e_spin = np.zeros((nspin, nspin))
for i in range(nspin):
    for j in range(nspin):
        if not i%2 == j%2:
            continue
        p = i // 2
        q = j // 2
        h1e_spin[i, j] = h1e[p, q]

# 双电子积分扩展到自旋轨道
h2e_spin = np.zeros((nspin, nspin, nspin, nspin))
for i in range(nspin):
    for j in range(nspin):
        for k in range(nspin):
            for l in range(nspin):
                if (i % 2) == (k % 2) and (j % 2) == (l % 2):
                    p, q, r, s = i//2, j//2, k//2, l//2
                    h2e_spin[i, j, k, l] = eri[p, r, q, s]

# 4. 生成所有可能的电子组态
nelec = mol.nelectron  # H₂的电子数为2
configs = list(itertools.combinations(range(nspin), nelec))
n_configs = len(configs)

# 5. 构建哈密顿矩阵
H = np.zeros((n_configs, n_configs))

for i in range(n_configs):
    A = configs[i]
    for j in range(i, n_configs):
        B = configs[j]
        diff = set(A).symmetric_difference(B)
        if len(diff) == 0:
            # 对角元
            h = 0.0
            # 单电子部分
            for p in A:
                h += h1e_spin[p, p]
            # 双电子部分
            for p, q in itertools.combinations(A, 2):
                h += 0.5 * (h2e_spin[p, q, p, q] - h2e_spin[p, q, q, p])
            H[i, j] = H[j, i] = h
        elif len(diff) == 2:
            # 单激发
            removed = list(set(A) - set(B))
            added = list(set(B) - set(A))
            r, s = removed[0], added[0]
            
            # 计算相位
            a_list = list(A)
            a_list.remove(r)
            insert_pos = bisect.bisect_left(a_list, s)
            temp = a_list[:insert_pos] + [s] + a_list[insert_pos:]
            assert temp == list(B), "Phase error"
            original_pos = A.index(r)
            new_pos = B.index(s)
            # phase = (-1) ** (original_pos + new_pos)
            phase = (-1) ** (new_pos - original_pos)
            
            # 单电子贡献
            h_single = h1e_spin[r, s] * phase
            # 双电子贡献
            sum_double = 0.0
            for t in A:
                if t != r:
                    sum_double += (h2e_spin[r, t, s, t] - h2e_spin[r, t, t, s]) * phase
            H_ij = h_single + sum_double
            H[i, j] = H[j, i] = H_ij
        elif len(diff) == 4:
            # 双激发
            removed = sorted(set(A) - set(B))
            added = sorted(set(B) - set(A))
            r1, r2 = removed
            s1, s2 = added
            
            # 计算相位
            temp = list(A)
            temp.remove(r1)
            temp.remove(r2)
            temp.extend([s1, s2])
            temp.sort()
            assert temp == list(B), "Phase error"
            
            # 获取原始位置和新位置
            pos_r1 = A.index(r1)
            pos_r2 = A.index(r2)
            pos_s1 = B.index(s1)
            pos_s2 = B.index(s2)
            
            # 简化的相位计算（可能需要更精确的方法）
            # phase = (-1) ** (pos_r1 + pos_r2 + pos_s1 + pos_s2)
            phase = (-1) ** ( pos_s1 + pos_s2 - pos_r1 - pos_r2)
            
            # 双电子贡献
            H_ij = (h2e_spin[r1, r2, s1, s2] - h2e_spin[r1, r2, s2, s1]) * phase
            H[i, j] = H[j, i] = H_ij

# 6. 对角化哈密顿矩阵
eigvals, eigvecs = np.linalg.eigh(H)
print("Full CI ground state energy:", eigvals[0])

converged SCF energy = -1.06610864931794
Full CI ground state energy: -1.943943284920306


## 极简FCI

In [40]:
import numpy as np
from pyscf import gto, scf, ao2mo
import itertools
import bisect

# 1. 定义分子并进行HF计算
mol = gto.M(atom='H 0 0 0; H 0 0 1.0', basis='sto-3g')
mf = scf.RHF(mol).run()

# 2. 获取分子轨道积分
h1e = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)  # 单电子积分
nmo = h1e.shape[0]
# eri = ao2mo.kernel(mol, mf.mo_coeff)  # 双电子积分 (物理 notation)
# eri = eri.reshape(nmo, nmo, nmo, nmo)
eri = ao2mo.kernel(mf.mol, mo_coeff,aosym=1)
eri = eri.reshape(nmo, nmo, nmo, nmo)
# 3. 扩展到自旋轨道
nspin = nmo * 2  # 自旋轨道数目

# 单电子积分扩展到自旋轨道
h1e_spin = np.zeros((nspin, nspin))
for i in range(nspin):
    for j in range(nspin):
        if i % 2 == j % 2:
            p = i // 2
            q = j // 2
            h1e_spin[i, j] = h1e[p, q]

# 双电子积分扩展到自旋轨道
h2e_spin = np.zeros((nspin, nspin, nspin, nspin))
for i in range(nspin):
    for j in range(nspin):
        for k in range(nspin):
            for l in range(nspin):
                if (i % 2) == (k % 2) and (j % 2) == (l % 2):
                    p, q, r, s = i//2, j//2, k//2, l//2
                    h2e_spin[i, j, k, l] = eri[p, r, q, s]

# 4. 生成所有可能的电子组态
nelec = mol.nelectron  # H₂的电子数为2
configs = list(itertools.combinations(range(nspin), nelec))
n_configs = len(configs)

converged SCF energy = -1.06610864931794


In [48]:
from pyscf.fci import cistring

In [49]:
cistring.make_strings(range(norb),nelec//2)

array([1, 2])

In [54]:
cistring.gen_linkstr_index(orb_list = range(norb),nocc = nelec//2)

array([[[0, 0, 0, 1],
        [1, 0, 1, 1]],

       [[1, 1, 1, 1],
        [0, 1, 0, 1]]], dtype=int32)